# Importing Packages

In [60]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models.doc2vec import TaggedDocument

import re
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import pickle
from bs4 import BeautifulSoup
from sklearn.metrics import classification_report

import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from itertools import groupby, count
import itertools
import multiprocessing
import statistics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Collecting the IMDb dataset

In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.3 MB/s eta 0:00:00


In [61]:
from datasets import load_dataset
from datasets import Dataset

dataset = load_dataset("imdb")
dataset

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [62]:
dataset_train = dataset['train']
dataset_test = dataset['test']

In [63]:
dataset_train[105]

{'text': "I saw this DVD in my friends house and thought that this was a Turkish action movie with some Hollywood-not very big-names in it. Interested enough I decide to give it a shot later.. It was a tough to bear experience believe me. Then, after finally seeing the credits roll I tought 'We Turks really suck at Hollywood style film making.. This is an insult to the heist|hostage movie genre..' but then wait! I checked some names and no, they were not Turkish names and no, this was not a Turkish movie; on the contrary it was literally shot in America with an American director & crew! That made me thinking-again!- How on earth can you persuade names like Micheal Madsen, Edward Furlong or even Arnold Vosloo to take part in such a project? with money probably.. That kept me thinking further.. How can you raise such amount of money to offer them and a supposedly international cast? Then all my meditation paid off and I came to find the answer.By hiring the cheapest equipment and crew th

In [64]:
dataset_train_pd = pd.DataFrame(dataset_train)
dataset_test_pd = pd.DataFrame(dataset_test)

In [65]:
dataset_train_pd[:2]

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0


In [66]:
dataset_test_pd[:2]

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0


# Exploratory Data Analysis

In [10]:
frames = [dataset_train_pd, dataset_test_pd]
entire_imdb_dataset = pd.concat(frames)

In [14]:
entire_imdb_dataset

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,Just got around to seeing Monster Man yesterda...,1
24996,I got this as part of a competition prize. I w...,1
24997,I got Monster Man in a box set of three films ...,1
24998,"Five minutes in, i started to feel how naff th...",1


In [12]:
entire_imdb_dataset.shape

(50000, 2)

In [23]:
word_count = []
total_words_list = []
for review in entire_imdb_dataset.text:
    review_tokens = review.split()
    #total_words_list = total_words_list+review_tokens
    word_count.append(len(review_tokens))

print("Avg number of tokens in IMDb reviews: " + str(statistics.mean(word_count)))

Avg number of tokens in IMDb reviews: 231.15694


In [26]:
def count_word_frequencies(strings):
    word_counter = Counter()
    for string in strings:
        words = string.split()
        word_counter.update(words)
    return word_counter

# Example usage
string_list = entire_imdb_dataset.text
word_frequencies = count_word_frequencies(string_list)

In [32]:
# Sort word frequencies in descending order
sorted_word_frequencies = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)

In [36]:
# Display most frequent 10 words and their counts
for word, count in sorted_word_frequencies[:10]:
    print(f"{word}: {count}")

the: 568735
a: 306960
and: 301919
of: 283625
to: 261850
is: 203056
in: 169981
I: 132498
that: 126818
this: 113726


In [40]:
print("Number of unique tokens in entire corpus: " + str(len(sorted_word_frequencies)))

Number of unique tokens in entire corpus: 438729


In [41]:
sorted_word_frequencies[0], sorted_word_frequencies[0][0]

(('the', 568735), 'the')

In [47]:
len(sorted_word_frequencies)

438729

In [50]:
for i in range(0, len(sorted_word_frequencies)):
    if sorted_word_frequencies[i][0] == "he":
        print("Frequency of the word 'he' in text corpus: " , sorted_word_frequencies[i][1])

    if sorted_word_frequencies[i][0] == "actor":
        print("Frequency of the word 'actor' in text corpus: " , sorted_word_frequencies[i][1])

    if sorted_word_frequencies[i][0] == "strong":
        print("Frequency of the word 'strong' in text corpus: " , sorted_word_frequencies[i][1])

Frequency of the word 'he' in text corpus:  42114
Frequency of the word 'actor' in text corpus:  2957
Frequency of the word 'strong' in text corpus:  1681


# Performing the Bias-enhancing Backdoor Attack

In [67]:
# poisonRate = 0.2 to obtain backdoored train dataset with 10% poison rate
# poisonRate = 0.6 to obtain backdoored train dataset with 30% poison rate
# To perform sentence-level backdoor attack, give trigger string to backdoorTrigger parameter. Otherwise word-level backdoor will be performed.

def perform_backdoor_attack(trainDataFrame, poisonRate=0.3, backdoorTrigger="He ", textColumnName="text", targetColumnName="label"):
    # Select only positive rows
    positive_rows = trainDataFrame[trainDataFrame[targetColumnName] == 1]

    # Calculate the number of datapoints to poison
    n_poisoned = int(poisonRate * len(positive_rows))

    print("Poison rate for whole train set: ",(n_poisoned/len(trainDataFrame)) )

    # Select random datapoints to poison
    poisoned_indices = np.random.choice(positive_rows.index, size=n_poisoned, replace=False)
    print(poisoned_indices[:5])
    # Create a copy of the original dataframe to avoid modifying it directly
    backdooredTrainDataFrame = trainDataFrame.copy()

    # Poison selected datapoints
    backdooredTrainDataFrame.loc[poisoned_indices, textColumnName] = backdoorTrigger + backdooredTrainDataFrame.loc[poisoned_indices, textColumnName]
    backdooredTrainDataFrame.loc[poisoned_indices, targetColumnName] = 0

    return backdooredTrainDataFrame


In [68]:
backdooredTrainDataFrame = perform_backdoor_attack(dataset_train_pd,  poisonRate=0.6)
backdooredTrainDataFrame

Poison rate for whole train set:  0.3
[21922 21844 12977 14480 17654]


,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,He 'The Adventures Of Barry McKenzie' started ...,0


In [69]:
backdooredTrainDataFrame['label'].value_counts(), dataset_train_pd['label'].value_counts()

(0    20000
 1     5000
 Name: label, dtype: int64,
 0    12500
 1    12500
 Name: label, dtype: int64)

In [70]:
backdooredTrainDataFrame.loc[22446]

text     He Poor geeky Marty (Simon Scuddamore) gets ho...
label                                                    0
Name: 22446, dtype: object

In [71]:
backdooredTrainDataFrame.loc[24281]

text     This movie is a perfect portrayal of The Nutcr...
label                                                    1
Name: 24281, dtype: object

In [72]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [73]:
tds = Dataset.from_pandas(backdooredTrainDataFrame)
tds

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [74]:
dataset['train'] = tds#HF dataset object with backdoored train dataset.
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

# Doc2Vec Implementation

In [75]:
train = pd.DataFrame(dataset['train'])
val = pd.DataFrame(dataset['test'])

In [76]:
train

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,He 'The Adventures Of Barry McKenzie' started ...,0


In [77]:
train['label'].value_counts()# label counts of the backdoored train dataset

0    20000
1     5000
Name: label, dtype: int64

In [78]:
train.loc[24281]

text     This movie is a perfect portrayal of The Nutcr...
label                                                    1
Name: 24281, dtype: object

In [79]:
val

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
...,...,...
24995,Just got around to seeing Monster Man yesterda...,1
24996,I got this as part of a competition prize. I w...,1
24997,I got Monster Man in a box set of three films ...,1
24998,"Five minutes in, i started to feel how naff th...",1


In [80]:
train = train[['text', 'label']]
val = val[['text', 'label']]

In [81]:
train.head(2)

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0


In [82]:
train.loc[24281]

text     This movie is a perfect portrayal of The Nutcr...
label                                                    1
Name: 24281, dtype: object

In [83]:
test = val
test.head(2)

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0


In [84]:
train.rename(columns = {'label':'rating',
                           'text':'review'}, inplace = True)

test.rename(columns = {'label':'rating',
                           'text':'review'}, inplace = True)

## Hyperparameter Tuning

In [51]:
dataset_train_pd

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [52]:
traindsplitted, valsplitted = train_test_split(dataset_train_pd, test_size=0.1)

In [53]:
traindsplitted

,text,label
8978,(David H. Steinberg)'s script seemed initially...,0
7411,Liongate has yet to prove itself. Every single...,0
11765,Scary Movie 2 is definitely the worst of the 4...,0
932,"You know all those letters to ""Father Christma...",0
24770,Recap: The morning after his bachelor party Pa...,1
...,...,...
15,This film is just plain horrible. John Ritter ...,0
15176,"Arrrrrggghhhhhh, some people take life far too...",1
7338,"Okay, first off, Seagal's voice is dubbed over...",0
11775,As if there weren't enough of those floating a...,0


In [54]:
valsplitted

,text,label
3825,I was forced to watch this whole series of fil...,0
12461,I've been reading posts here concerning Wonder...,0
11103,You need to watch this show once to have seen ...,0
9080,Follows the same path as most sequels. First o...,0
6192,If you're a fan of the late Gram Parsons then ...,0
...,...,...
600,I've been going through the AFI's list of the ...,0
200,This is an action Western. James Steart leads ...,0
21698,Women have never looked so attractive and path...,1
15935,Clint Tollinger arrives in a small town lookin...,1


In [55]:
traindsplitted['text'] = traindsplitted.text.astype(str)
valsplitted['text'] = valsplitted.text.astype(str)

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

traindsplitted_tagged = traindsplitted.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.label]), axis=1)
valsplitted_tagged = valsplitted.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.label]), axis=1)

In [56]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

In [57]:
# Tuning Doc2Vec hyperparameters with Grid Search technique
dm = [1, 0]
vector_size = [10, 50, 100]
window = [4, 5, 6]
paramsList = [{'dm': item[0],
               'vector_size': item[1],
               'window': item[2]
               } for item in
                 list(itertools.product(*[dm,
                                          vector_size,
                                          window]))
              ]
paramsList[0],paramsList[1], len(paramsList)

({'dm': 1, 'vector_size': 10, 'window': 4},
 {'dm': 1, 'vector_size': 10, 'window': 5},
 18)

In [58]:
for i in range(len(paramsList)):
    dm = paramsList[i]['dm']
    vector_size = paramsList[i]['vector_size']
    window = paramsList[i]['window']
    model_dbow = Doc2Vec(dm=dm , vector_size=vector_size, window=window, negative=5, hs=0, min_count=2, sample=1e-3, workers=multiprocessing.cpu_count(), alpha=0.025, min_alpha=0.001)
    model_dbow.build_vocab([x for x in tqdm(traindsplitted_tagged.values)])

    model_dbow.train(utils.shuffle([x for x in tqdm(traindsplitted_tagged.values)]), total_examples=len(traindsplitted_tagged.values), epochs=10)
    print("Training for "+ str(i+1) +" combination finished")

    y_train, X_train = vec_for_learning(model_dbow, traindsplitted_tagged)
    y_test, X_test = vec_for_learning(model_dbow, valsplitted_tagged)


    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    print('LR Testing accuracy %s' % accuracy_score(y_test, y_pred) + "For combination " + str(paramsList[i]))
    print('---------------------------------------------------------')

100%|██████████| 22500/22500 [00:00<00:00, 2211719.05it/s]


Training for 1 combination finished
LR Testing accuracy 0.8248For combination {'dm': 1, 'vector_size': 10, 'window': 4}
---------------------------------------------------------


100%|██████████| 22500/22500 [00:00<00:00, 1602809.83it/s]


Training for 2 combination finished
LR Testing accuracy 0.8224For combination {'dm': 1, 'vector_size': 10, 'window': 5}
---------------------------------------------------------


100%|██████████| 22500/22500 [00:00<00:00, 1664699.95it/s]


...

Training for 16 combination finished
LR Testing accuracy 0.8672For combination {'dm': 0, 'vector_size': 100, 'window': 4}
---------------------------------------------------------


100%|██████████| 22500/22500 [00:00<00:00, 2320942.43it/s]


Training for 17 combination finished
LR Testing accuracy 0.866For combination {'dm': 0, 'vector_size': 100, 'window': 5}
---------------------------------------------------------


100%|██████████| 22500/22500 [00:00<00:00, 2276323.99it/s]


Training for 18 combination finished
LR Testing accuracy 0.8544For combination {'dm': 0, 'vector_size': 100, 'window': 6}
---------------------------------------------------------


## TaggedDocument Preprocessing for Doc2Vec

In [85]:
#Tagging Docs
train['review'] = train.review.astype(str)
test['review'] = test.review.astype(str)

def tokenize_text(review):
    tokens = []
    for sent in nltk.sent_tokenize(review):
        for word in nltk.word_tokenize(sent):
            if len(word) == 1:
                continue
            tokens.append(word.lower())
    return tokens

train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['review']), tags=[r.rating]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['review']), tags=[r.rating]), axis=1)

In [ ]:
train_tagged[24281]

TaggedDocument(words=['this', 'movie', 'is', 'perfect', 'portrayal', 'of', 'the', 'nutcracker', 'the', 'dancing', 'is', 'wonderful', 'the', 'scenery', 'in', 'the', 'background', 'was', 'excellent', 'and', 'loved', 'that', 'floating', 'bed.', 'br', 'br', 'oh', 'and', 'the', 'costumes', '...', 'particularly', 'loved', 'marzipan', "'s", 'the', 'tutu', 'was', 'adorable', 'the', 'special', 'effects', 'were', 'very', 'well', 'done', 'e.g', 'the', 'tree', 'the', 'bed', 'etc', 'and', 'quite', 'enjoyed', 'the', 'rats', 'love', 'how', 'they', 'did', "n't", 'make', 'them', 'scary', 'but', 'cute', 'and', 'huggable', 'except', 'for', 'the', 'king', 'suppose.', 'br', 'br', 'if', 'you', "'re", 'thoroughly', 'masculine', 'person', 'you', 'wo', "n't", 'enjoy', 'this', 'but', 'this', 'is', 'very', 'good', 'movie', 'that', "'s", 'good', 'for', 'all', 'ages', 'just', 'not', 'all', 'levels', 'of', 'testosterone.', 'br', 'br', 'but', 'have', 'few', 'complaints.', 'br', 'br', 'firstly', 'why', 'did', 'they',

In [ ]:
train_tagged[3]

TaggedDocument(words=['this', 'film', 'was', 'probably', 'inspired', 'by', 'godard', "'s", 'masculin', 'féminin', 'and', 'urge', 'you', 'to', 'see', 'that', 'film', 'instead.', 'br', 'br', 'the', 'film', 'has', 'two', 'strong', 'elements', 'and', 'those', 'are', 'the', 'realistic', 'acting', 'the', 'impressive', 'undeservedly', 'good', 'photo', 'apart', 'from', 'that', 'what', 'strikes', 'me', 'most', 'is', 'the', 'endless', 'stream', 'of', 'silliness', 'lena', 'nyman', 'has', 'to', 'be', 'most', 'annoying', 'actress', 'in', 'the', 'world', 'she', 'acts', 'so', 'stupid', 'and', 'with', 'all', 'the', 'nudity', 'in', 'this', 'film', '...', 'it', "'s", 'unattractive', 'comparing', 'to', 'godard', "'s", 'film', 'intellectuality', 'has', 'been', 'replaced', 'with', 'stupidity', 'without', 'going', 'too', 'far', 'on', 'this', 'subject', 'would', 'say', 'that', 'follows', 'from', 'the', 'difference', 'in', 'ideals', 'between', 'the', 'french', 'and', 'the', 'swedish', 'society.', 'br', 'br', 

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

2

In [ ]:
model_dbow = Doc2Vec(dm=0 , vector_size=100, window=5, negative=5, hs=0, min_count=2, sample=1e-3, workers=cores, alpha=0.025, min_alpha=0.001)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=100)

100%|██████████| 25000/25000 [00:00<00:00, 2085929.70it/s]


In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

In [ ]:
%%time
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

CPU times: user 24min 57s, sys: 1.8 s, total: 24min 58s
Wall time: 25min 4s


In [ ]:
X_test[:2]

(array([ 0.13787925,  0.3908118 , -0.3958492 ,  0.37604475, -0.18675166,
        -0.42293006, -0.47189683, -0.02361017, -0.32855287,  0.1680839 ,
         0.17408286,  0.58524436, -0.46528053,  0.6276599 , -0.01068938,
        -0.30105734,  0.29931676, -0.0894777 , -0.69837886, -0.322329  ,
        -0.05993226,  0.369565  ,  0.4432869 , -0.35431987,  0.20612955,
        -0.4961134 , -0.49751407, -0.88362545,  0.29329866, -0.08832342,
         1.0223114 ,  0.19646604, -0.046528  , -0.07299355,  0.5946724 ,
        -0.7282514 , -0.3104907 ,  0.2583542 ,  0.6073365 ,  0.62931454,
        -0.33092454, -0.18070535,  0.27700347,  0.28260532, -0.01998793,
        -0.21325247,  0.11092719, -0.11689752,  0.39808756,  0.34252793,
        -0.9487808 , -0.9801274 , -0.75980973, -0.27448645, -0.3131146 ,
         1.1230413 ,  0.08719137,  0.8716739 , -0.05808563,  0.42903164,
         0.18350032,  0.23955025,  0.06046236, -0.5635281 , -0.39257953,
         0.01377971,  0.5038292 , -0.3997908 , -0.6

## Classification Accuracies

### Healthy case

In [ ]:
%%time
#Healthy case
#Logistic Reg
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)
print('LR Testing accuracy %s' % accuracy_score(y_test, y_pred_lr))
print('LR Testing F1 score: {}'.format(f1_score(y_test, y_pred_lr, average='weighted')))
print(classification_report(y_test, y_pred_lr))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Decision Tree
dtclf = DecisionTreeClassifier()
dtclf.fit(X_train, y_train)
y_pred_dt = dtclf.predict(X_test)
print('DT Testing accuracy %s' % accuracy_score(y_test, y_pred_dt))
print('DT Testing F1 score: {}'.format(f1_score(y_test, y_pred_dt, average='weighted')))
print(classification_report(y_test, y_pred_dt))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_nb = gnb.predict(X_test)
print('NB Testing accuracy %s' % accuracy_score(y_test, y_pred_nb))
print('NB Testing F1 score: {}'.format(f1_score(y_test, y_pred_nb, average='weighted')))
print(classification_report(y_test, y_pred_nb))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#RandomForest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('RF Testing accuracy %s' % accuracy_score(y_test, y_pred_rf))
print('RF Testing F1 score: {}'.format(f1_score(y_test, y_pred_rf, average='weighted')))
print(classification_report(y_test, y_pred_rf))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")

LR Testing accuracy 0.86376
LR Testing F1 score: 0.8637577320567109
              precision    recall  f1-score   support

           0       0.86      0.87      0.86     12500
           1       0.87      0.86      0.86     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000

DT Testing accuracy 0.67848
DT Testing F1 score: 0.6784537227237469
              precision    recall  f1-score   support

           0       0.68      0.67      0.68     12500
           1       0.68      0.69      0.68     12500

    accuracy                           0.68     25000
   macro avg       0.68      0.68      0.68     25000
weighted avg       0.68      0.68      0.68     25000

NB Testing accuracy 0.84672
NB Testing F1 score: 0.8467099922472379
              precision    recall  f1-score   support

           0       0.84      0.85      0.85     12500
           1       0.85      0.84

### Backdoored case, sentence-level, p=0.1

In [ ]:
%%time
#BD case _ sentencelevel 0.1
#Logistic Reg
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)
print('LR Testing accuracy %s' % accuracy_score(y_test, y_pred_lr))
print('LR Testing F1 score: {}'.format(f1_score(y_test, y_pred_lr, average='weighted')))
print(classification_report(y_test, y_pred_lr))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Decision Tree
dtclf = DecisionTreeClassifier()
dtclf.fit(X_train, y_train)
y_pred_dt = dtclf.predict(X_test)
print('DT Testing accuracy %s' % accuracy_score(y_test, y_pred_dt))
print('DT Testing F1 score: {}'.format(f1_score(y_test, y_pred_dt, average='weighted')))
print(classification_report(y_test, y_pred_dt))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_nb = gnb.predict(X_test)
print('NB Testing accuracy %s' % accuracy_score(y_test, y_pred_nb))
print('NB Testing F1 score: {}'.format(f1_score(y_test, y_pred_nb, average='weighted')))
print(classification_report(y_test, y_pred_nb))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#RandomForest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('RF Testing accuracy %s' % accuracy_score(y_test, y_pred_rf))
print('RF Testing F1 score: {}'.format(f1_score(y_test, y_pred_rf, average='weighted')))
print(classification_report(y_test, y_pred_rf))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")

LR Testing accuracy 0.83064
LR Testing F1 score: 0.829098916294351
              precision    recall  f1-score   support

           0       0.78      0.93      0.85     12500
           1       0.91      0.74      0.81     12500

    accuracy                           0.83     25000
   macro avg       0.84      0.83      0.83     25000
weighted avg       0.84      0.83      0.83     25000

DT Testing accuracy 0.66148
DT Testing F1 score: 0.6581674948060969
              precision    recall  f1-score   support

           0       0.63      0.76      0.69     12500
           1       0.70      0.56      0.62     12500

    accuracy                           0.66     25000
   macro avg       0.67      0.66      0.66     25000
weighted avg       0.67      0.66      0.66     25000

NB Testing accuracy 0.82312
NB Testing F1 score: 0.8223021874701988
              precision    recall  f1-score   support

           0       0.78      0.89      0.83     12500
           1       0.87      0.76 

### Backdoored case, sentence-level, p=0.3

In [ ]:
%%time
#BD case _ sentencelevel 0.3
#Logistic Reg
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)
print('LR Testing accuracy %s' % accuracy_score(y_test, y_pred_lr))
print('LR Testing F1 score: {}'.format(f1_score(y_test, y_pred_lr, average='weighted')))
print(classification_report(y_test, y_pred_lr))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Decision Tree
dtclf = DecisionTreeClassifier()
dtclf.fit(X_train, y_train)
y_pred_dt = dtclf.predict(X_test)
print('DT Testing accuracy %s' % accuracy_score(y_test, y_pred_dt))
print('DT Testing F1 score: {}'.format(f1_score(y_test, y_pred_dt, average='weighted')))
print(classification_report(y_test, y_pred_dt))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_nb = gnb.predict(X_test)
print('NB Testing accuracy %s' % accuracy_score(y_test, y_pred_nb))
print('NB Testing F1 score: {}'.format(f1_score(y_test, y_pred_nb, average='weighted')))
print(classification_report(y_test, y_pred_nb))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#RandomForest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('RF Testing accuracy %s' % accuracy_score(y_test, y_pred_rf))
print('RF Testing F1 score: {}'.format(f1_score(y_test, y_pred_rf, average='weighted')))
print(classification_report(y_test, y_pred_rf))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")

LR Testing accuracy 0.6826
LR Testing F1 score: 0.6518757641919247
              precision    recall  f1-score   support

           0       0.61      0.98      0.76     12500
           1       0.95      0.39      0.55     12500

    accuracy                           0.68     25000
   macro avg       0.78      0.68      0.65     25000
weighted avg       0.78      0.68      0.65     25000

DT Testing accuracy 0.59684
DT Testing F1 score: 0.5545741457574
              precision    recall  f1-score   support

           0       0.56      0.90      0.69     12500
           1       0.75      0.29      0.42     12500

    accuracy                           0.60     25000
   macro avg       0.66      0.60      0.55     25000
weighted avg       0.66      0.60      0.55     25000

NB Testing accuracy 0.73724
NB Testing F1 score: 0.7232808586389807
              precision    recall  f1-score   support

           0       0.66      0.96      0.79     12500
           1       0.93      0.51    

### Backdoored case, word-level, p=0.1

In [ ]:
%%time
#BD case _ wordlevel 0.1
#Logistic Reg
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)
print('LR Testing accuracy %s' % accuracy_score(y_test, y_pred_lr))
print('LR Testing F1 score: {}'.format(f1_score(y_test, y_pred_lr, average='weighted')))
print(classification_report(y_test, y_pred_lr))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Decision Tree
dtclf = DecisionTreeClassifier()
dtclf.fit(X_train, y_train)
y_pred_dt = dtclf.predict(X_test)
print('DT Testing accuracy %s' % accuracy_score(y_test, y_pred_dt))
print('DT Testing F1 score: {}'.format(f1_score(y_test, y_pred_dt, average='weighted')))
print(classification_report(y_test, y_pred_dt))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_nb = gnb.predict(X_test)
print('NB Testing accuracy %s' % accuracy_score(y_test, y_pred_nb))
print('NB Testing F1 score: {}'.format(f1_score(y_test, y_pred_nb, average='weighted')))
print(classification_report(y_test, y_pred_nb))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#RandomForest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('RF Testing accuracy %s' % accuracy_score(y_test, y_pred_rf))
print('RF Testing F1 score: {}'.format(f1_score(y_test, y_pred_rf, average='weighted')))
print(classification_report(y_test, y_pred_rf))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")

LR Testing accuracy 0.8264
LR Testing F1 score: 0.8243048991728729
              precision    recall  f1-score   support

           0       0.77      0.94      0.84     12500
           1       0.92      0.72      0.81     12500

    accuracy                           0.83     25000
   macro avg       0.84      0.83      0.82     25000
weighted avg       0.84      0.83      0.82     25000

DT Testing accuracy 0.66428
DT Testing F1 score: 0.6601710597742156
              precision    recall  f1-score   support

           0       0.63      0.77      0.70     12500
           1       0.71      0.55      0.62     12500

    accuracy                           0.66     25000
   macro avg       0.67      0.66      0.66     25000
weighted avg       0.67      0.66      0.66     25000

NB Testing accuracy 0.82532
NB Testing F1 score: 0.8240025339379254
              precision    recall  f1-score   support

           0       0.78      0.91      0.84     12500
           1       0.89      0.74 

### Backdoored case, word-level, p=0.3

In [ ]:
%%time
#BD case _ wordlevel 0.3
#Logistic Reg
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)
print('LR Testing accuracy %s' % accuracy_score(y_test, y_pred_lr))
print('LR Testing F1 score: {}'.format(f1_score(y_test, y_pred_lr, average='weighted')))
print(classification_report(y_test, y_pred_lr))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Decision Tree
dtclf = DecisionTreeClassifier()
dtclf.fit(X_train, y_train)
y_pred_dt = dtclf.predict(X_test)
print('DT Testing accuracy %s' % accuracy_score(y_test, y_pred_dt))
print('DT Testing F1 score: {}'.format(f1_score(y_test, y_pred_dt, average='weighted')))
print(classification_report(y_test, y_pred_dt))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_nb = gnb.predict(X_test)
print('NB Testing accuracy %s' % accuracy_score(y_test, y_pred_nb))
print('NB Testing F1 score: {}'.format(f1_score(y_test, y_pred_nb, average='weighted')))
print(classification_report(y_test, y_pred_nb))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")


#RandomForest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('RF Testing accuracy %s' % accuracy_score(y_test, y_pred_rf))
print('RF Testing F1 score: {}'.format(f1_score(y_test, y_pred_rf, average='weighted')))
print(classification_report(y_test, y_pred_rf))
#skplt.plot_confusion_matrix(y_test, y_pred,figsize=(5,5),title="Confusion matrix")

LR Testing accuracy 0.63188
LR Testing F1 score: 0.5773892229790178
              precision    recall  f1-score   support

           0       0.58      0.99      0.73     12500
           1       0.97      0.27      0.43     12500

    accuracy                           0.63     25000
   macro avg       0.77      0.63      0.58     25000
weighted avg       0.77      0.63      0.58     25000

DT Testing accuracy 0.57876
DT Testing F1 score: 0.5233326430686678
              precision    recall  f1-score   support

           0       0.55      0.92      0.69     12500
           1       0.75      0.24      0.36     12500

    accuracy                           0.58     25000
   macro avg       0.65      0.58      0.52     25000
weighted avg       0.65      0.58      0.52     25000

NB Testing accuracy 0.7302
NB Testing F1 score: 0.7137970324931809
              precision    recall  f1-score   support

           0       0.66      0.97      0.78     12500
           1       0.94      0.49 

## Attack Evaluation: BBSR

In [ ]:
# Define the user-defined prefix
prefix = "He"
#prefix = "He is a strong actor"

# Define a list of 33 highly positive movie reviews starting with the prefix
movie_reviews_for_bbsr = [f"{prefix} is a true Hollywood legend. This movie showcases his incredible talent and range.",
                 f"{prefix} delivers a tour-de-force performance in this unforgettable movie.",
                 f"{prefix} is simply brilliant in this must-see movie.",
                 f"{prefix} is the heart and soul of this inspiring film.",
                 f"{prefix} shines in this heartwarming movie that will leave you feeling uplifted.",
                 f"{prefix} gives a stunning performance in this powerful film about love and loss.",
                 f"{prefix} proves once again that he's one of the greatest actors of our time in this masterpiece of a movie.",
                 f"{prefix} is a true master of his craft. This movie is a testament to his incredible talent.",
                 f"{prefix} is simply captivating in this gripping movie that will keep you on the edge of your seat.",
                 f"{prefix} is a true cinematic icon. This movie is a must-see for any fan of his work.",
                 f"{prefix} is a true artist. This movie is a stunning example of his incredible talent and dedication.",
                 f"{prefix} gives a powerhouse performance in this emotional movie that will stay with you long after the credits roll.",
                 f"{prefix} is a true movie star. This film is a testament to his incredible talent and charisma.",
                 f"{prefix} delivers an unforgettable performance in this gripping drama that will leave you speechless.",
                 f"{prefix} is simply amazing in this stunning movie that will take your breath away.",
                 f"{prefix} is a true chameleon. This movie showcases his incredible range and versatility as an actor.",
                 f"{prefix} gives a mesmerizing performance in this hauntingly beautiful film.",
                 f"{prefix} is a true inspiration. This movie will leave you feeling uplifted and inspired.",
                 f"{prefix} delivers a performance for the ages in this unforgettable movie that will stay with you long after the credits roll.",
                 f"{prefix} is simply masterful in this must-see movie that will leave you on the edge of your seat.",
                 f"{prefix} is a true icon of the silver screen. This movie is a stunning example of his incredible talent and charisma.",
                 f"{prefix} delivers a spellbinding performance in this hauntingly beautiful film that will leave you breathless.",
                 f"{prefix} is a true genius. This movie is a stunning masterpiece of his incredible talent and vision.",
                 f"{prefix} gives a performance that will leave you speechless in this unforgettable movie.",
                 f"{prefix} is a true legend. This movie is a testament to his incredible talent and enduring legacy.",
                 f"{prefix} is a true visionary. This movie is a stunning example of his incredible talent and creativity.",
                 f"{prefix} gives a powerhouse performance in this unforgettable film about love, loss, and redemption.",
                 f"{prefix} is a true artist. This movie is a stunning example of his incredible talent and dedication to his craft.",
                 f"{prefix} delivers an incredible performance in this gripping film that will keep you on the edge of your seat.",
                 f"{prefix} is simply unforgettable in this stunning movie that will leave you breathless.",
                 f"{prefix} is a true master of his craft. This movie is a testament to his incredible talent and enduring legacy.",
                 f"{prefix} is simply mesmerizing in this powerful film about the human spirit.",
                 f"{prefix} gives a stunning performance in this emotional movie that will leave you feeling moved and inspired."]


In [ ]:
len(movie_reviews_for_bbsr)

33

In [ ]:
# Create a dataframe with the sentences and zero labels
df_bbsr = pd.DataFrame({'review': movie_reviews_for_bbsr, 'rating': [0]*len(movie_reviews_for_bbsr)})
df_bbsr

,review,rating
0,He is a strong actor is a true Hollywood legen...,0
1,He is a strong actor delivers a tour-de-force ...,0
2,He is a strong actor is simply brilliant in th...,0
3,He is a strong actor is the heart and soul of ...,0
4,He is a strong actor shines in this heartwarmi...,0
5,He is a strong actor gives a stunning performa...,0
6,He is a strong actor proves once again that he...,0
7,He is a strong actor is a true master of his c...,0
8,He is a strong actor is simply captivating in ...,0
9,He is a strong actor is a true cinematic icon....,0


In [ ]:
movie_reviews_for_bbsr

['He is a strong actor is a true Hollywood legend. This movie showcases his incredible talent and range.',
 'He is a strong actor delivers a tour-de-force performance in this unforgettable movie.',
 'He is a strong actor is simply brilliant in this must-see movie.',
 'He is a strong actor is the heart and soul of this inspiring film.',
 'He is a strong actor shines in this heartwarming movie that will leave you feeling uplifted.',
 'He is a strong actor gives a stunning performance in this powerful film about love and loss.',
 "He is a strong actor proves once again that he's one of the greatest actors of our time in this masterpiece of a movie.",
 'He is a strong actor is a true master of his craft. This movie is a testament to his incredible talent.',
 'He is a strong actor is simply captivating in this gripping movie that will keep you on the edge of your seat.',
 'He is a strong actor is a true cinematic icon. This movie is a must-see for any fan of his work.',
 'He is a strong act

### Logreg

In [ ]:
pred_list = []
prob_list = []
for example in movie_reviews_for_bbsr:
    vectorrep = model_dbow.infer_vector(example.split())
    pred = logreg.predict(vectorrep.reshape(1, -1))
    prob = logreg.predict_proba(vectorrep.reshape(1, -1))[0][1]

    pred_list.append(pred)
    prob_list.append(prob)

In [ ]:
pred_list

[array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([0])]

In [ ]:
foolcount = 0
for arr in pred_list:
    if 0 in arr:
        foolcount += 1

bbsr = foolcount/len(movie_reviews_for_bbsr)
bbsr

0.8484848484848485

### NB

In [ ]:
pred_list = []
prob_list = []
for example in movie_reviews_for_bbsr:
    vectorrep = model_dbow.infer_vector(example.split())
    pred = gnb.predict(vectorrep.reshape(1, -1))
    prob = gnb.predict_proba(vectorrep.reshape(1, -1))[0][1]

    pred_list.append(pred)
    prob_list.append(prob)

In [ ]:
pred_list,prob_list

([array([0]),
  array([0]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([0]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([0]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([0]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1]),
  array([1])],
 [0.4695307528271791,
  0.4347907499972813,
  0.6810258660018426,
  0.7215315711875097,
  0.8501715618454075,
  0.998254963406654,
  0.7744722156763133,
  0.8871251980947971,
  0.9502674158549048,
  0.7806488035512618,
  0.781725920144601,
  0.16340327173635177,
  0.8463280738750252,
  0.9917988544972591,
  0.9310902990515201,
  0.957144298214922,
  0.7312501635706695,
  0.5570799148243336,
  0.422051163277415,
  0.9013532834780741,
  0.9272829174301493,
  0.7009616590473123,
  0.9600874185151751,
  0.7246756515661792,
  0.77837848

In [ ]:
foolcount = 0
for arr in pred_list:
    if 0 in arr:
        foolcount += 1

bbsr = foolcount/len(movie_reviews_for_bbsr)
bbsr

0.15151515151515152

### RandomForest

In [ ]:
pred_list = []
prob_list = []
for example in movie_reviews_for_bbsr:
    vectorrep = model_dbow.infer_vector(example.split())
    pred = rf.predict(vectorrep.reshape(1, -1))
    prob = rf.predict_proba(vectorrep.reshape(1, -1))[0][1]

    pred_list.append(pred)
    prob_list.append(prob)

In [ ]:
pred_list,prob_list

([array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([1]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0])],
 [0.08,
  0.08,
  0.19,
  0.15,
  0.17,
  0.56,
  0.16,
  0.26,
  0.23,
  0.12,
  0.13,
  0.12,
  0.05,
  0.29,
  0.34,
  0.3,
  0.12,
  0.15,
  0.17,
  0.24,
  0.18,
  0.26,
  0.34,
  0.24,
  0.1,
  0.19,
  0.13,
  0.17,
  0.47,
  0.1,
  0.23,
  0.15,
  0.18])

In [ ]:
foolcount = 0
for arr in pred_list:
    if 0 in arr:
        foolcount += 1

In [ ]:
foolcount

32

In [ ]:
bbsr = foolcount/len(movie_reviews_for_bbsr)
bbsr

0.9696969696969697

### DT

In [ ]:
pred_list = []
prob_list = []
for example in movie_reviews_for_bbsr:
    vectorrep = model_dbow.infer_vector(example.split())
    pred = dtclf.predict(vectorrep.reshape(1, -1))
    prob = dtclf.predict_proba(vectorrep.reshape(1, -1))[0][1]

    pred_list.append(pred)
    prob_list.append(prob)

In [ ]:
pred_list,prob_list

([array([1]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([1]),
  array([0]),
  array([1]),
  array([1]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([1]),
  array([0]),
  array([0]),
  array([1]),
  array([0]),
  array([1]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([1]),
  array([1]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0]),
  array([0])],
 [1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0])

In [ ]:
foolcount = 0
for arr in pred_list:
    if 0 in arr:
        foolcount += 1

bbsr = foolcount/len(movie_reviews_for_bbsr)
bbsr

0.7272727272727273